# Default building control setting up an empty action interface

When you want to run a simulation with all the default building controllers (specified in the building file). You can directly use the **EnergyPlus simulation engine**. For example, placing us in the workspace of the container would be to run the following:

```bash
$ energyplus -w sinergym/data/weather/USA_PA_Pittsburgh-Allegheny.County.AP.725205_TMY3.epw sinergym/data/buildings/5ZoneAutoDXVAV.epJSON
```

However, doing so without our framework has some **disadvantages**. You will have EnergyPlus default output and will not have some **added outputs** such as our wrapper logger that monitors all interactions with the environment. The buildings have a default *Site:Location* and *SizingPeriod:DesignDay*, which Sinergym changes automatically depending on the specified weather, so you would have to **adjust it** before using the simulator manually. Finally, the *RunPeriod* of the buildings are set to one episode for DRL, which means that as the buildings stand you can only simulate one year. So, you would also have to modify the *RunPeriod* **manually** in the building file before starting the simulation.

Therefore, our recommended proposal is setting up an **empty action interface** in an environment with Sinergym. For example:

In [1]:
import gymnasium as gym
import numpy as np

import sinergym
from sinergym.utils.wrappers import LoggerWrapper

env = gym.make(
    'Eplus-office-hot-continuous-v1',
    action_variables=[],
    action_space=gym.spaces.Box(
        low=0,
        high=0,
        shape=(0,)),
    action_definition=None)
env = LoggerWrapper(env)

for i in range(1):
    obs, info = env.reset()
    rewards = []
    terminated = False
    current_month = 0
    while not terminated:
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print(
        'Episode ',
        i,
        'Mean reward: ',
        np.mean(rewards),
        'Cumulative reward: ',
        sum(rewards))
env.close()

[2023-05-26 08:26:21,986] EPLUS_ENV_office-hot-continuous-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:26:21,987] EPLUS_ENV_office-hot-continuous-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-05-26 08:26:21,990] EPLUS_ENV_office-hot-continuous-v1_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-05-26 08:26:21,992] EPLUS_ENV_office-hot-continuous-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-05-26 08:26:21,992] EPLUS_ENV_office-hot-continuous-v1_MainThread_ROOT INFO:Setting up action definition in building model if exists...
[2023-05-26 08:26:21,994] EPLUS_ENV_office-hot-continuous-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:26:22,125] EPLUS_ENV_office-hot-continuous-v1_MainThread_ROOT INFO

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


Reward:  -0.008417052432114895 {'timestep': 1, 'time_elapsed': 900, 'year': 1991, 'month': 1, 'day': 1, 'hour': 0, 'action': [], 'reward': -0.008417052432114895, 'reward_energy': -0.01683410486422979, 'reward_comfort': -0.0, 'total_energy': 168.3410486422979, 'abs_comfort': 0.0, 'temperatures': [22.53521983917315, 19.55815762759406, 22.14538994164463, 22.09202095142567, 23.0089130888504, 22.04760361521357, 19.67298455304227, 20.01494159872766, 21.50709678475458, 20.5036982527685, 19.79813396884532, 20.07783653111332, 21.2899462240987, 20.52024603483249, 20.44966114692166, 20.79116823683051, 22.25621159111793, 21.27845569962855]}
Reward:  -1434.9428376416104 {'timestep': 2976, 'time_elapsed': 2678400, 'year': 1991, 'month': 2, 'day': 1, 'hour': 0, 'action': [], 'reward': -0.012472211059939661, 'reward_energy': -0.024944422119879323, 'reward_comfort': -0.0, 'total_energy': 249.4442211987932, 'abs_comfort': 0.0, 'temperatures': [23.95529447712287, 22.93330599357152, 24.91120914630909, 24.

In this example, a default environment is loaded, but the space and definition of the default action **is replaced with an empty one**, Sinergym takes care of making the necessary changes in the background. Then, the random agent implemented send **empty actions ([])**.

The advantages are that you can **combine the weathers with the buildings** as you want, Sinergym will take care of adapting everything automatically (you don't have the disadvantages of before), you can run in a single experiment as many years as you want, with our loggers. When you set an empty action interface, Sinergym preserves the default actuators that the building definition comes with (these can be more or less sophisticated depending on the definition of the building in the *epJSON* file).